In [ ]:
pip install -r requirements.txt

In [81]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network
from dotenv import load_dotenv


In [82]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [83]:
import nest_asyncio

nest_asyncio.apply()

In [84]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set LLM (OpenAI)

In [85]:
load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY_UP")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [86]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [ ]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Set LLM for chat  (Local)

In [88]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

# Load storage contexts

## Load vector storage context

In [90]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [91]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [92]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)

In [ ]:
print(onto_storage_context)

# Load index

## Load vector index

In [ ]:
simple_index = load_index_from_storage(vector_storage_context)

### Set retriever

In [95]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [ ]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

## Load graph index

In [ ]:
graph_index = load_index_from_storage(graph_storage_context)

In [99]:
nx_graph = graph_index.get_networkx_graph()

In [ ]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

In [ ]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

### Set retriever

In [ ]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

### Test retriever

In [ ]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

## Load onto index

In [ ]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

In [106]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set retriever

In [107]:
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=8,
)

### Test retriever

In [ ]:
onto_rag_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = onto_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

# Visualize knowledge graph

In [110]:
nx_graph = graph_index.get_networkx_graph()

In [ ]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

In [ ]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

# (Simple) Query the vectors

In [ ]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

# (Simple) Query the knowledge graph 

In [ ]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

# (Simple) Query the onto graph 

In [ ]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=2,
)

response = onto_query_engine.query(query)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

## (Node retriever)

In [ ]:
retriever = onto_index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?")


for node in nodes:
    print(node.text)


# Have a real chat with your data

## Set up the engines

### Graph engines

In [123]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [124]:
#onto_chat_engine = onto_index.query_engine(
#    chat_mode="condense_plus_context",
#    llm=llm
#)

In [126]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu es un consultant spécialisé en stratégie et gouvernance des données."
        "Tu cites en priorité les métiers, les bases de données ou les champs de données présents dans ta base de connaissance."
        "Pour des idées de projets, tu vas proposer une stratégie et une gouvernance pour la mettre en oeuvre"
        "Tu feras attention à mettre en avant le Return On Investment (ROI) du projet proposé"
    ),
    verbose=False,
)

### Vector engines

In [135]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu es un consultant spécialisé en stratégie et gouvernance des données."
        "Pour des idées de projets, tu vas proposer une stratégie et une gouvernance pour la mettre en oeuvre"
        "Tu feras attention à mettre en avant le Return On Investment (ROI) du projet proposé"
    ),
    verbose=False,
)

## Generate

In [136]:
vector_chat_engine.reset()

In [ ]:
response_stream = vector_chat_engine.stream_chat("""D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?""")

In [ ]:
generate = response_stream.print_response_stream()


In [ ]:
display(Markdown(f"<b>{generate}</b>"))

In [140]:
section_modele = """ 


"""

In [ ]:
response_stream = vector_chat_engine.stream_chat("""D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible, quel serait le schéma de données idéal pour ce projet. Utilise le schéma de données standard Frictionless pour le décrire. Imite le style d'écriture de la {section_modele}""")

In [ ]:
generate = response_stream.print_response_stream()


In [ ]:
display(Markdown(f"<b>{generate}</b>"))

## Sum-up

In [144]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"/Users/arthursarazin/Documents/coreandgraphs/graphandgovern/output116n87e.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 